In [1]:
%pip install langchain-google-genai langchain-core pydantic python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal
from pydantic import BaseModel, Field

load_dotenv()

True

In [4]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
parser = StrOutputParser()

In [5]:
# --- [원본 Cell 3: 간단한 체인] ---
print("--- [Cell 3: Simple Chain (model | parser)] ---")

#"모델을 실행하고(1단계), 그 결과(AIMessage)를 받아, 문자열로 파싱(변환)하는(2단계)" 두 단계를 하나로 묶은 새로운 '체인'을 정의
chain = model | parser

messages_cell1 = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화하라."),
    HumanMessage(content="안녕? 저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?"),
]
print(chain.invoke(messages_cell1))

--- [Cell 3: Simple Chain (model | parser)] ---
개스톤, 안녕하세요. 저녁 식사 말씀이신가요? 아, 죄송하지만 저는 오늘 저녁에 할 일이 좀 많아서요. 특히 읽어야 할 책이 산더미 같거든요. 조용한 저녁이 저에게는 더 맞을 것 같아요.


In [6]:
# --- [원본 Cell 4, 5: 프롬프트 템플릿 체인] ---
print("\n--- [Cell 5: Prompt + Model + Parser Chain] ---")
system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕? 저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"


--- [Cell 5: Prompt + Model + Parser Chain] ---


In [7]:
# (원본 Cell 4)
prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("user", human_template),
])

In [12]:
# (원본 Cell 5)
chain = prompt_template | model | parser

print(chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
}))

야수님, 안녕하세요.

저녁 식사 말씀이신가요? 글쎄요... 지금 막 읽던 책이 너무 흥미로워서요. 하지만... 야수님께서 저녁 식사를 제안해주시다니, 감사한 일이죠.

좋아요, 그럼 저녁 식사 시간에 맞춰 내려갈게요. 혹시... 식사 전에 잠시 도서관에 들러도 될까요? 새로운 이야기가 저를 기다리고 있을 것 같아서요.


In [8]:
# --- [원본 Cell 7: Pydantic (Structured Output) 체인] ---
print("\n--- [Cell 7: Structured Output Chain] ---")
class Adlib(BaseModel):
    """스토리 설정과 사용자 입력에 반응하는 대사를 만드는 클래스"""
    answer: str = Field(description="스토리 설정과 사용자와의 대화 기록에 따라 생성된 대사")
    main_emotion: Literal["기쁨", "분노", "슬픔", "공포", "냉소", "불쾌", "중립"] = Field(description="대사의 주요 감정")
    main_emotion_intensity: float = Field(description="대사의 주요 감정의 강도 (0.0 ~ 1.0)")

# .with_structured_output은 Gemini 모델에서도 동일하게 작동합니다.
structured_llm = model.with_structured_output(Adlib)
adlib_chain = prompt_template | structured_llm

print(adlib_chain.invoke({
    "story": "미녀와 야수",
    "character_a": "벨",
    "character_b": "개스톤",
    "activity": "저녁"
}))


--- [Cell 7: Structured Output Chain] ---
answer='오, 개스톤. 죄송하지만 저는 저녁에 할 일이 많아요. 읽어야 할 책들도 저를 기다리고 있구요.' main_emotion='불쾌' main_emotion_intensity=0.4
